In [1]:
import pandas as pd
import ezdxf
import numpy as np

In [2]:
path = "../calc/tho_res_plots.xlsx"
sheet_name = ['lpa_0633', 'lpa_0568', 'lpa_0366', 'lpa_0300', 'lpa_0101']

DF_nodes = pd.read_excel(path, sheet_name='knoten')[['X [m]', 'Y [m]', 'Z [m]']]

In [3]:
def read_data(path, sheet_name):
    DF = pd.read_excel(path, sheet_name=sheet_name)[['Knoten', 'My  [kNm]', 'Tx  [kNm]']].dropna()
    #transformation
    DF['Knoten'] = DF['Knoten'].astype(int)*-1 -1
    #Merge with nodes coordinates
    DF_res = DF.merge(DF_nodes, left_on='Knoten', right_index=True)
    #rename
    columns_rename = ['knoten', 'My', 'Tx', 'x', 'y', 'z']
    DF_res.columns = columns_rename
    #reset index
    DF_res=DF_res.reset_index()

    return DF_res

def save_dxf(Data, dxf_name, moment):

    # Extract rows for DXF
    x = Data['x']
    y = Data['y']
    z = Data[moment]/5

    # Generate consistent pairs (0-1, 2-3, ...)
    index_pairs = [(i, i + 1) for i in range(0, len(x), 2)]

    # Create a new DXF document
    doc = ezdxf.new()

    # Add a new modelspace
    msp = doc.modelspace()



    # Add 3D polyline to the DXF
    points = list(zip(x, y, z))

    # Add line segments for each pair
    for start, end in index_pairs:
        points = [(x[start],y[start],z[start]), (x[end],y[end],z[end])]
        msp.add_polyline3d(points, dxfattribs={'layer': '3D Line'})
        

    # Save to file
    output_path = dxf_name+'_'+moment+".dxf"
    doc.saveas(output_path)

    print(f"DXF file saved as {output_path}")

In [4]:
for sheet in sheet_name:    
    DF = read_data(path = path, sheet_name=sheet)
    save_dxf(DF, sheet, 'My')
    save_dxf(DF, sheet, 'Tx')

DXF file saved as lpa_0633_My.dxf
DXF file saved as lpa_0633_Tx.dxf
DXF file saved as lpa_0568_My.dxf
DXF file saved as lpa_0568_Tx.dxf
DXF file saved as lpa_0366_My.dxf
DXF file saved as lpa_0366_Tx.dxf
DXF file saved as lpa_0300_My.dxf
DXF file saved as lpa_0300_Tx.dxf
DXF file saved as lpa_0101_My.dxf
DXF file saved as lpa_0101_Tx.dxf
